# <center> <img src="figs/LogoUFSCar.jpg" alt="Logo UFScar" width="110" align="left"/>  <br/> <center>Universidade Federal de São Carlos (UFSCar)<br/><font size="4"> Departamento de Computação, campus Sorocaba</center></font>
</p>

<font size="4"><center><b>Disciplina: Aprendizado de Máquina</b></center></font>
  
<font size="3"><center>Prof. Dr. Tiago A. Almeida</center></font>

## <center>Projeto Final</center>

**Aluno**: Eduardo Garcia do Nascimento

**RA/CPF**: 22008732800


---
### Carregamento dos dados

Nesta seção é feita a carga dos atributos em um dataframe só, ou seja, os três datasets são lidos e concatenados para que a redução de atributos leve em conta o que existe de melhor em todos eles.


In [1]:
# -*- coding: utf-8 -*-

# Caminho dos arquivos
FILES_DIRECTORY = "data"

import numpy as np 
import pandas as pd 
import os
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif
from scripts import utils

if __name__ == '__main__':
                       
    # importa o arquivo e guarda em um dataframe do Pandas
    set1_dataset  = pd.read_csv(os.path.join(FILES_DIRECTORY, 'set1.csv'), sep=',', low_memory=False)
    set2_dataset  = pd.read_csv(os.path.join(FILES_DIRECTORY, 'set2.csv'), sep=',', low_memory=False) 
    set3_dataset  = pd.read_csv(os.path.join(FILES_DIRECTORY, 'set3.csv'), sep=',', low_memory=False)
    train_dataset = pd.read_csv(os.path.join(FILES_DIRECTORY, 'train.csv'), sep=',')
    test_dataset  = pd.read_csv(os.path.join(FILES_DIRECTORY, 'test.csv'), sep=',')
    
    # Renomeia colunas concatenando o setX antes de fazer o merge para identificá-las posteriormente
    cols = set1_dataset.columns
    for col in cols:
        set1_dataset = set1_dataset.rename(columns={col:'set1_'+col})
        
    cols = set2_dataset.columns
    for col in cols:
        set2_dataset = set2_dataset.rename(columns={col:'set2_'+col})
    
    cols = set3_dataset.columns
    for col in cols:
        set3_dataset = set3_dataset.rename(columns={col:'set3_'+col})

    # Concatena os datasets em somente um dataset único
    frames = [ set1_dataset, set2_dataset, set3_dataset ]
    input_dataset = pd.concat(frames, axis=1)
    
    print('A base de dados inicial combinada tem %d amostras com %d atributos.' % (input_dataset.shape[0],
                                                                                 input_dataset.shape[1]))
    
        
   
    utils.beep(1, 400)

KeyboardInterrupt: 

---
### Pré-processamento e seleção de atributos

Nesta seção são feitas limpezas da base de dados como:

* Remoção de atributos sem variância;
* Tratamento de outliers e dados nulos;
* Seleção dos atributos que terão maior valor para o algoritmo de classificação.

In [ ]:
# Remove os atributos que são constantes e não oferecem nenhum valor aos algoritmos de classificação
variance_mask = VarianceThreshold().fit(input_dataset).get_support()
input_dataset = input_dataset.iloc[:,variance_mask]
print('Atributos removidos por baixa variância: %d' % np.sum(~variance_mask))

# Tratamento de outliers e entradas nulas
df = input_dataset.copy()
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
outliers_mask = (df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))
print("Número de outliers substituídos por valores nulos:", np.sum(np.sum(outliers_mask)))
df[outliers_mask] = np.nan
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
input_dataset.loc[:,:] = imp_mean.fit_transform(input_dataset)

# Normalização dos dados entre 0 e 1
input_dataset.loc[:,:] = MinMaxScaler().fit_transform(input_dataset)

# Adiciona as classes junto ao dataset de atributos para fazer a seleção de atributos
input_dataset['classe'] = np.nan
input_dataset.loc[train_dataset['Id'].values,'classe'] = train_dataset['Class'].values

# Salva uma cópia do dataset completo que será usada para predição de novas amostras.
backup_dataset = input_dataset.copy()

feature_cols = list(input_dataset.columns)
feature_cols.remove('classe')

# Seleciona os melhores atritubos para treinametno do algoritmo de classificação
print('Selecionando melhores features....................................')
selector = SelectKBest(f_classif, k=6).fit(
    input_dataset.loc[(input_dataset['classe'] == -1) | (input_dataset['classe'] == 1), feature_cols].values,
    input_dataset.loc[(input_dataset['classe'] == -1) | (input_dataset['classe'] == 1),'classe'].values)
features_mask = selector.get_support()
features_mask = np.append(features_mask, True)
input_dataset = input_dataset.iloc[:,features_mask]

print('\n\n\n\nBreve avaliação das primeiras amostras')
display(input_dataset.head(10))

print('Avaliação do descritivo do dataset que permite ter uma ideia mais realista dos dados')
display(input_dataset.describe())

utils.beep(1, 500)

---
### Análise exploratória

Nesta seção são exibidas informações do resultado após os dados serem pré-processados e os atributos selecionados. Dentre as ferramentas para análise exploratória que serão utilizados estão:

* Descritivo resumido da base.
* Análises de covariância e correlação.
* Matriz de disperação entre todos os atributos selecionados. 
* Diagramas de violino para visualização dos quartis e outliers como uma variação aos diagramas de caixa.
* Gráfico de dispersão com a dimensionalidade reduzida a somente 2 atributos.

In [ ]:
# Análise do balanceamento das classes
print('Análise e visualização dos dados:')
ax = sns.countplot(x='classe', data=input_dataset, label="Contagem")

N,U,P = input_dataset.classe.value_counts()
print('Número de posts comuns:', N)
print('Número de posts não reconhecidos (podem ser phishing ou não):', U)
print('Número de posts phishing:', P)
plt.show()

# Análise da matriz scatter pra que entendamos a relação entre os atributos
print('Análise da matriz de dispersão')
sns.pairplot(input_dataset.loc[(input_dataset['classe'] == -1) | (input_dataset['classe'] == 1)], hue='classe', height=3.5);
plt.show()

# matrizes de covariancia e correlação
print('Análise das matrizes de covariância ')
df_covariance = input_dataset.iloc[:,:-1].cov()
df_correlation = input_dataset.iloc[:,:-1].corr()
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20, 8))
plt.title('Covariância')
sns.heatmap(df_covariance, annot=True, xticklabels=df_correlation.columns, yticklabels=df_correlation.columns, ax=ax1)
plt.title('Correlação')
sns.heatmap(df_correlation, annot=True, xticklabels=df_correlation.columns, yticklabels=df_correlation.columns, ax=ax2)
plt.show()

# Montagem do dataset de teste para envio para o Kaggle
K = input_dataset.loc[test_dataset.iloc[:,:].values.T[0]]
K = K.drop('classe', axis=1).values

# Diagramas de caixa
plt.figure(figsize=(20,10))
data = pd.melt(input_dataset.loc[(input_dataset['classe'] == -1) | (input_dataset['classe'] == 1)],
               id_vars="classe", var_name="features", value_name='value')
sns.boxplot(x='features', y='value', hue='classe', data=data)
plt.show()

# Diagramas de violino
plt.figure(figsize=(20,10))
sns.violinplot(x='features', y='value', hue='classe', data=data, split=True, inner="quartile")
plt.show()

# Separação de atributos e classe para 
utils.printPCA(input_dataset.loc[((input_dataset['classe'] == -1) | (input_dataset['classe'] == 1))].drop('classe', axis=1).values,
               input_dataset.loc[((input_dataset['classe'] == -1) | (input_dataset['classe'] == 1)), 'classe'].values)    

from sklearn.decomposition import PCA
pca = PCA(2)  # project from 64 to 2 dimensions
projected = pca.fit_transform(input_dataset.loc[((input_dataset['classe'] == -1) | (input_dataset['classe'] == 1))].drop('classe', axis=1).values)
data = pd.DataFrame(data=projected, columns=['a', 'b'])
data['classe'] = input_dataset.loc[((input_dataset['classe'] == -1) | (input_dataset['classe'] == 1)), 'classe'].values

if int(sns.__version__[2:4]) < 11:
    print('Para uma melhor experiência, atualize a seaborn para a versão 11 ou superior')
    sns.jointplot(data=data, x="a", y="b")
else:
    sns.jointplot(data=data, x="a", y="b", kind="kde", palette='Spectral', hue='classe')

utils.beep(1, 600)   

---
### Preparo dos dados e experimentos para encontrar os melhores hiperparâmetros

Nesta seção os dados são separados em duas partes: treino e testes. Esta estratégia foi utilizada para posterior comparação com a validação de modelos utilizandos K-folds.
Para encontrar os melhores hiperparâmetros foi utilizada a classe GridSearchCV e devido o seu alto custo computacional e não existência da necessidade de executá-las sempre, a sua chamada é condicionada às variáveis booleanas evaluate_svm_hiperparameters, evaluate_rfc_hiperparameters e evaluate_lrc_hiperparameters serem verdadeiras.

In [ ]:
from sklearn.model_selection import train_test_split
from scripts import utils

X_total = input_dataset.loc[(input_dataset['classe'] == -1) | (input_dataset['classe'] == 1)].drop('classe', axis=1).values
y_total = input_dataset.loc[((input_dataset['classe'] == -1) | (input_dataset['classe'] == 1)), 'classe'].values

# Utilize as flags abaixo somente para avaliação dos hiperparâmetros pois elas demoram muito pra serem executadas
evaluate_svm_hiperparameters = False # Busca os melhores parâmetros para as máquinas de vetores de suporte
evaluate_rfc_hiperparameters = False # Busca os melhores parâmetros para as florestas aleatórias
evaluate_knn_hiperparameters = False # Busca os melhores parâmetros para o KNN

print('Removendo amostras outliers')
X, y = utils.remove_outliers(X_total, y_total)

print(X.shape, y.shape)
print('Separando a base em treino e teste')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=0, shuffle=True, stratify=y)

scores = ['balanced_accuracy', 'f1', 'roc_auc']

if evaluate_svm_hiperparameters == True:
    utils.find_best_svm(X,y, scores)
    
if evaluate_rfc_hiperparameters == True:    
    utils.find_best_rfc(X, y, scores)
        
if evaluate_knn_hiperparameters == True:
    utils.find_best_knn(X, y, scores)

utils.beep(1, 700)

---
### Experimento

Nesta seção, o experimento deve ser conduzido, utilizando os protocolos experimentais ensinados no curso e executando os métodos inteligentes

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import  plot_confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.semi_supervised import LabelSpreading
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.utils import resample

X_bal, y_bal = X, y
X_train_bal, y_train_bal = X_train, y_train


scoring=['roc_auc', 'f1', 'f1_micro', 'f1_macro', 'f1_weighted', 'accuracy', 'balanced_accuracy', 'precision', 'recall']

results = pd.DataFrame()

model_list = [
    ['SVM poly', svm.SVC(kernel='poly', class_weight='balanced', decision_function_shape='ovr', probability=True,random_state=1), 0],
   # ['SVM linear', svm.SVC(kernel='linear', C=100, class_weight='balanced', decision_function_shape='ovr', probability=True,random_state=1), 0],
   # ['SVM rbf', svm.SVC(kernel='rbf', C=1000, gamma=0.0001, class_weight='balanced', decision_function_shape='ovr', probability=True,random_state=1), 0],
   # ['SVM rbf_ra', svm.SVC(kernel='rbf', C=10, gamma='scale', class_weight='balanced', decision_function_shape='ovr', probability=True,random_state=1), 0],
   # ['SVM rbf_ba', svm.SVC(kernel='rbf', C=400, gamma='scale', class_weight='balanced', decision_function_shape='ovr', probability=True,random_state=1), 0],
   # ['SVM rbf_f1', svm.SVC(kernel='rbf', C=600, gamma=0.001, class_weight='balanced', decision_function_shape='ovr', probability=True,random_state=1), 0],
    ['Random Forest', RandomForestClassifier(max_depth=2, class_weight='balanced', random_state=1), 0],
    ['Logistic Regression', LogisticRegression(random_state=1, class_weight='balanced', max_iter=15000), 0],
    ['Multinomial NB', MultinomialNB(), 0],
    ['KNN', KNeighborsClassifier(n_neighbors=1, leaf_size=4, weights='distance'), 0],
   # ['AdaBoost', AdaBoostClassifier(n_estimators=100, random_state=0), 0],
   # ['GradBoost', GradientBoostingClassifier(random_state=0), 0],
   # ['KNN3', KNeighborsClassifier(n_neighbors=1, leaf_size=4, weights='distance'), 0],
   # ['KNN8', KNeighborsClassifier(n_neighbors=70, leaf_size=1, weights='distance'), 0],
]

print(np.sum(y_train_bal==-1),np.sum(y_train_bal==1))
print(np.sum(y_bal==-1),np.sum(y_bal==1))

# Pré balanceamento dos dados utilizando a técnica de oversampling
X_train_bal, y_train_bal = utils.balance_classes(X_train_bal, y_train_bal)
X_bal, y_bal = utils.balance_classes(X, y)

utils.find_best_knn(X_bal, y_bal, scores)

print(np.sum(y_train_bal==-1),np.sum(y_train_bal==1))
print(np.sum(y_bal==-1),np.sum(y_bal==1))

semi_model = svm.SVC(kernel='poly', class_weight='balanced', decision_function_shape='ovr', probability=True,random_state=1)
 # População dos dados não classificados com o melhor classificador encontrado com os resultados mais confiáveis
if False:
    samples_mask = (backup_dataset.classe!=-1)&(backup_dataset.classe!=1)&(backup_dataset.classe!=0)
    samples = backup_dataset[samples_mask].iloc[:,features_mask].drop('classe', axis=1).values
    X_semi, y_semi, proba_mask = utils.add_samples(semi_model,X_bal, y_bal, samples)

    print('Número de amostras acrescentadas ao dataset de treinamento:', X_semi.shape[0], 
                                                                          np.sum(y_semi==-1), np.sum(y_semi==1))
    X_train_bal = np.concatenate([X_train_bal, X_semi], axis=0)
    y_train_bal = np.concatenate([y_train_bal, y_semi], axis=0)
    X_bal = np.concatenate([X_bal, X_semi], axis=0)
    y_bal = np.concatenate([y_bal, y_semi], axis=0)

if False:
    #X_train_bal, y_train_bal = utils.balance_classes(X_train_bal, y_train_bal)
    #X_bal, y_bal = utils.balance_classes(X_bal, y_bal)
    X_bal, y_bal = utils.downsample(X_bal, y_bal)
    # População dos dados não classificados com o melhor classificador encontrado com os resultados mais confiáveis
    samples = samples[~proba_mask]
    X_semi, y_semi, proba_mask = utils.add_samples(semi_model, X_train_bal, y_train_bal, samples)
    print('Número de amostras acrescentadas ao dataset de treinamento:', X_semi.shape[0], 
                                                                          np.sum(y_semi==-1), np.sum(y_semi==1))
    X_train_bal = np.concatenate([X_train_bal, X_semi], axis=0)
    y_train_bal = np.concatenate([y_train_bal, y_semi], axis=0)
    X_bal = np.concatenate([X_bal, X_semi], axis=0)
    y_bal = np.concatenate([y_bal, y_semi], axis=0)

print(np.sum(y_train_bal==-1),np.sum(y_train_bal==1))
print(np.sum(y_bal==-1),np.sum(y_bal==1))

#X_train_bal, y_train_bal = utils.downsample(X_train_bal, y_train_bal)
#X_bal, y_bal = utils.downsample(X_bal, y_bal)
#X_train_bal, y_train_bal = utils.balance_classes(X_train_bal, y_train_bal)
#X_bal, y_bal = utils.balance_classes(X, y)

print(np.sum(y_train_bal==-1),np.sum(y_train_bal==1))
print(np.sum(y_bal==-1),np.sum(y_bal==1))

# Configura K folds estratificados, ou seja, mantendo as mesmas proporções entre classes
cv = StratifiedKFold(n_splits=4, random_state=1, shuffle=True)

for model in model_list:
    print(model[0] + '------------------------------------------------------------------------------------')
    y_pred = utils.evaluate_model(model[1], X_train_bal, y_train_bal, X_test, y_test)
    
    cv_results = cross_validate(model[1], X_bal, y_bal, scoring=scoring, return_train_score=True)

    for cv_result in cv_results:
        results.loc[model[0], cv_result] = np.mean(cv_results[cv_result])
    
    results.loc[model[0], 'TN'] = confusion_matrix(y_test, y_pred)[0][0]
    results.loc[model[0], 'TP'] = confusion_matrix(y_test, y_pred)[1][1]


utils.beep(1, 800)

---
### Análise dos Resultados

Nesta seção, os resultados devem ser exibidos e comparados, através de tabelas e gráficos

In [ ]:
def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    
    return ['background-color: yellow' if v else '' for v in is_max]

cols_filter = ['fit_time', 'score_time', 'test_roc_auc', 'test_f1', 
               'test_f1_micro', 'test_f1_macro', 'test_f1_weighted', 
              'test_accuracy', 'test_balanced_accuracy', 'train_balanced_accuracy', 
              'test_precision', 'test_recall', 'TN', 'TP']

results_highlited = results.loc[:,cols_filter].style.apply(highlight_max)



display(results_highlited)
utils.beep(3, 1000)

---
### Criação do arquivo de sumissão no Kaggle

Na etapa final, o arquivo submission.csv é criado para ser enviado ao Kaggle de acordo com os padrões pré-definidos pela proposta do desafio.

In [ ]:
submit_preference_score = 'SVM poly'

for model in model_list:
    if model[0]==submit_preference_score:
        best_model = model[1]
        best_model_name = model[0]

print('Criando arquivo de submissão para o modelo: ' + submit_preference_score)    
    
    
print('Printing model to submission.csv ###################################################################')
clf= best_model.fit(X_bal, y_bal)
y_pred_submission = clf.predict_proba(K)[:,1]
result = np.zeros((K.shape[0],2))
for i in range(K.shape[0]):
    result[i][0] = test_dataset.iloc[:,:].values.T[0][i]
    result[i][1] = y_pred_submission[i]
resultdf = pd.DataFrame(data=result, columns=["Id", "Predicted"])
resultdf['Id'] = resultdf['Id'].astype(int)
resultdf['Predicted'] = resultdf['Predicted'].round(decimals=5)
resultdf.to_csv('submission.csv', index=False, float_format='%.5f')
print('####################################################################################################')

utils.beep(5, 200)